# Problem 3

In [1]:
import random
import numpy as np
from skimage.io import imread
from skimage.color import gray2rgb
from skimage.measure import block_reduce
from scipy.misc import imresize
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


## Load the data

### Define the function to load the data

In [2]:
# Note : I have filtered out (deleted) the non_color image !!!
def filter_non_color(item):
    working_path = "/Users/vongola/Desktop/textminingHW2/IMAGE/" + item
    jpeg_files_path = glob.glob(os.path.join(working_path, '*.JPEG'))

    for path in jpeg_files_path:        
        temp_image = imread(path)
        if len(temp_image.shape) == 2: 
            print i 
            print path 

In [3]:
def load(item,class_item):
    
    working_path = "/Users/vongola/Desktop/textminingHW2/IMAGE/" + item
    jpeg_files_path = glob.glob(os.path.join(working_path, '*.JPEG'))
        
    for path in jpeg_files_path:        
        temp_image = imread(path)
        resize_image = imresize(temp_image, (64,64,3))
        temp = random.random()
        data.append(resize_image)
        label.append(class_item)

In [4]:
# Get the file path for all 8987 images
import os, glob
data = []
label = []

# load "n00017222" - plant - 0
#filter_non_color("n00017222")
load("n00017222",0)

# load "n09287968" - geological - 1
#filter_non_color("n09287968")
load("n09287968",1)

# load "n12992868" - fungus - 2
#filter_non_color("n12992868")
load("n12992868",2)

# load "n00523513" - sport - 3
#filter_non_color("n00523513")
load("n00523513",3)

# load "n00007846" - person - 4
#filter_non_color("n00007846")
load("n00007846",4)

# load "n00015388" - animal - 5
#filter_non_color("n00015388")
load("n00015388",5)

In [5]:
# Split the data into 20% test and 80% train
random.seed(666)
index = range(8973)
test_index = random.sample(range(8973),1800)
train_index = [x for x in index if x not in set(test_index)]

train_data = []
train_label = []
test_data = []
test_label = []

for index in train_index:
    train_data.append(data[index])
    train_label.append(label[index])

for index in test_index:
    test_data.append(data[index])
    test_label.append(label[index])
    
x_train = np.array(train_data)
x_train_label = np.array(train_label)
y_test = np.array(test_data)
y_test_label = np.array(test_label)

### Pre-processing the data for Autoencoder

In [6]:
x_train_normalize = x_train.astype('float32') / 255.
y_test_normalize = y_test.astype('float32') / 255.
x_train_normalize = np.reshape(x_train_normalize, (len(x_train_normalize), 64, 64, 3))  
y_test_normalize = np.reshape(y_test_normalize, (len(y_test_normalize), 64, 64, 3))  

## Convert the images to vectors - Autoencoder

Note : Choose the type 2 architucutre, as it got the least error among four types

In [25]:
input_img = Input(shape=(64, 64, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img) #64*64
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x) #64*64
x = MaxPooling2D((2, 2), padding='same')(x) #32*32
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x) #32*32
x = MaxPooling2D((2, 2), padding='same')(x) #16*16
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x) #8*8
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu',padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [26]:
from keras.callbacks import TensorBoard

autoencoder.fit(x_train_normalize, x_train_normalize,
                epochs=20,
                batch_size=128,
                shuffle=True,
                validation_data=(y_test_normalize, y_test_normalize),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Train on 7173 samples, validate on 1800 samples
Epoch 1/20
7173/7173 [==============================] - 99s - loss: 0.6719 - val_loss: 0.6785

In [27]:
vector_auto = encoder.predict(x_train_normalize)

## Convert the images to vectors - SVD

In [17]:
# Define the function to zero-mean the input
def zero_mean(data):
    
    m,n = data.shape
    new = np.ones((m,n))
    new = new * (data.sum()/float(m*n))
    
    return (data - new)

In [18]:
x_svd = []
for i in range(len(x_train)):
    x_svd.append(x_train[i].flatten())
    
x_svd = np.array(x_svd)
x_svd_zero = zero_mean(np.array(x_svd))

x_svd_test = []
for i in range(len(y_test)):
    x_svd_test.append(y_test[i].flatten())

x_svd_test = np.array(x_svd_test)
x_svd_test_zero = zero_mean(np.array(x_svd_test))

In [19]:
temp = np.append(x_svd_zero, x_svd_test_zero,axis = 0)

In [20]:
U, s, V = np.linalg.svd(temp, full_matrices=True)

In [21]:
vector_SVD = U[:7173,:50]

### Justify the chosen of k 

To keep the same dimensional as the other method, we set k to 50,100,200,500.And finally choose k = 50 with highest accuracy

## Convert the images to vectors - RGB

In [7]:
# Define the function to construct the RGB vector
def rgb_vector(vector):
    temp_vector = np.zeros(256).tolist()
    for index in vector:
        temp_vector[int(index)] += 1
    
    return temp_vector

In [8]:
# Define the function to convert image into vector
def to_vector(image):
    image = image.flatten()
    divide = len(image)/3
    
    part_1 = image[:divide]
    temp_1 = rgb_vector(part_1)
    
    part_2 = image[divide:divide*2]
    temp_1.extend(rgb_vector(part_2))
    
    
    part_3 = image[divide*2:]
    temp_1.extend(rgb_vector(part_3))
    
    return temp_1

In [9]:
temp = []
for i in range(len(x_train)):        
    temp.append(to_vector(x_train[i]))
    
vector_RGB = np.array(temp)

## Convert the images to vectors - HSV

In [23]:
from skimage import color
temp = []
for i in range(len(x_train)):        
    temp.append(to_vector(color.rgb2hsv(x_train[i]) * 255))
    
vector_HSV = np.array(temp)

## Compare Classification Result

In [10]:
from scipy.spatial import distance
from scipy.stats import pearsonr
from sklearn.metrics import confusion_matrix

In [11]:
# Define the function to convert origin shape(N,64,64,3) into (N,K)
def convert(origin):
    
    temp = []
    
    for i in range(len(origin)):
        temp.append(origin[i].flatten())
    
    return np.array(temp)

In [12]:
# Define the function to calculate the top 5 using Euclidean Distance
def cal_top5(test,train,metric):
    
    num_image = len(train)
    distance_value = []
    top5_index = []
    
    if metric == "E":
        for i in range(num_image):
            distance_value.append(distance.euclidean(train[i],test))
        
        distance_temp = distance_value[:]
        distance_temp.sort()
    
        for i in range(5):
            top5_index.append(x_train_label[distance_value.index(distance_temp[i])]) #get the label
    
        return top5_index
        
    else:
        for i in range(num_image):
            distance_value.append(pearsonr(train[i],test))
        
        
        distance_temp = distance_value[:]
        distance_temp.sort()
    
        for i in range(5):
            top5_index.append(x_train_label[distance_value.index(distance_temp[-1 - i])]) #get the label
    
        return top5_index

In [13]:
# Define the function to get the label among top 5 by majority vote
def get_label(list):
    
    count = [0,0,0,0,0,0]
    
    for value in list:
        count[value] += 1
    
    return count.index(max(count))      

In [14]:
# Define the function to get the accuracy
def get_accuracy(matrix):
    
    num_right = matrix[0][0] + matrix[1][1] + matrix[2][2] + matrix[3][3] + matrix[4][4] + matrix[5][5]
    num_total = matrix.sum()
    
    return round((num_right/float(num_total)) * 100,2)
    

In [15]:
# Define the function to calculate the accuracy and confusion matrix
def cal_acc(train,test,test_label,metric):
    
    # Get the top5
    test_top5 = []

    for i in range(len(test)):        
        test_top5.append(cal_top5(test[i],train,metric))
    
    # Get the label by majority vote
    pre_label = []

    for top in test_top5:
        pre_label.append(get_label(top))

    confusion_matrix_1 = confusion_matrix(test_label, pre_label)
    accuracy = get_accuracy(confusion_matrix_1)
    
    return confusion_matrix_1, accuracy
    

### Method 1 - Autoencoder

In [28]:
# Get the test set from autoencoder
test_predict_auto = encoder.predict(y_test_normalize)

# Get the test set into Normalize vector
test_predict_auto = convert(test_predict_auto)
x_train_auto = convert(vector_auto)

# Get the result
confusion_matrix_auto_Eu, accuracy_auto_Eu = cal_acc(x_train_auto, test_predict_auto, y_test_label, "E")
confusion_matrix_auto_Pe, accuracy_auto_Pe = cal_acc(x_train_auto, test_predict_auto, y_test_label, "P")

print "----------------------------- Result for Euclidean distance -----------------------------"
print confusion_matrix_auto_Eu
print "Accuracy is " + str(accuracy_auto_Eu) + "%"

print "----------------------------- Result for Pearson correlation coefficient -----------------------------"
print confusion_matrix_auto_Pe
print "Accuracy is " + str(accuracy_auto_Pe) + "%"

----------------------------- Result for Euclidean distance -----------------------------
[[140  51  30  20   7  16]
 [ 61 236   9  16   7  16]
 [ 90  59  61  15   7  18]
 [ 67 115  19 115   4  37]
 [ 59  84  27  26  41  24]
 [105  86  25  43  17  47]]
Accuracy is 35.56%
----------------------------- Result for Pearson correlation coefficient -----------------------------
[[127  51  39  19  12  16]
 [ 50 244   8  20  13  10]
 [ 79  55  69  15  13  19]
 [ 75 112  19 111  13  27]
 [ 45  77  22  30  58  29]
 [ 67  91  37  46  32  50]]
Accuracy is 36.61%


### Method 2 - SVD

In [22]:
# Get the test SVD

test_SVD = U[7173:,:50]

# Get the result
confusion_matrix_SVD_Eu, accuracy_SVD_Eu = cal_acc(vector_SVD, test_SVD, y_test_label, "E")
confusion_matrix_SVD_Pe, accuracy_SVD_Pe = cal_acc(vector_SVD, test_SVD, y_test_label, "P")

print "----------------------------- Result for Euclidean distance -----------------------------"
print confusion_matrix_SVD_Eu
print "Accuracy is " + str(accuracy_SVD_Eu) + "%"

print "----------------------------- Result for Pearson correlation coefficient -----------------------------"
print confusion_matrix_SVD_Pe
print "Accuracy is " + str(accuracy_SVD_Pe) + "%"


----------------------------- Result for Euclidean distance -----------------------------
[[146  55  26   9   3  25]
 [ 41 263   5  22   3  11]
 [ 92  70  56  13   2  17]
 [ 81 124  19  90   2  41]
 [ 57  81  37  34  30  22]
 [128  85  23  41   5  41]]
Accuracy is 34.78%
----------------------------- Result for Pearson correlation coefficient -----------------------------
[[139  36  39  15  14  21]
 [ 21 253   9  30  13  19]
 [ 72  37  83  19  22  17]
 [ 51  87  23 142  22  32]
 [ 33  59  35  34  70  30]
 [ 73  65  37  55  43  50]]
Accuracy is 40.94%


### Method 3 - RGB 

In [16]:
# Get the test set into RGB vector
temp = []
for i in range(len(y_test)):        
    temp.append(to_vector(y_test[i]))
    
test_RGB = np.array(temp)

# Get the result
confusion_matrix_RGB_Eu, accuracy_RGB_Eu = cal_acc(vector_RGB, test_RGB, y_test_label, "E")
confusion_matrix_RGB_Pe, accuracy_RGB_Pe = cal_acc(vector_RGB, test_RGB, y_test_label, "P")

print "----------------------------- Result for Euclidean distance -----------------------------"
print confusion_matrix_RGB_Eu
print "Accuracy is " + str(accuracy_RGB_Eu) + "%"

print "----------------------------- Result for Pearson correlation coefficient -----------------------------"
print confusion_matrix_RGB_Pe
print "Accuracy is " + str(accuracy_RGB_Pe) + "%"

----------------------------- Result for Euclidean distance -----------------------------
[[127  19  82   9  13  14]
 [ 84 158  44  16  10  33]
 [ 90  14 110  17   6  13]
 [ 95  50  56  96  12  48]
 [ 62  30  69  30  46  24]
 [100  52  67  32  18  54]]
Accuracy is 32.83%
----------------------------- Result for Pearson correlation coefficient -----------------------------
[[103  35  75  19  17  15]
 [ 48 196  31  23  14  33]
 [ 84  30 110   5   7  14]
 [ 66  82  41 114  14  40]
 [ 52  50  56  29  55  19]
 [ 78  70  60  33  27  55]]
Accuracy is 35.17%


### Method 4 - HSV

In [24]:
# Get the test set into HSV vector
temp = []
for i in range(len(y_test)):
    temp.append(to_vector(color.rgb2hsv(y_test[i]) * 255))

test_HSV = np.array(temp)  

# Get the result
confusion_matrix_HSV_Eu, accuracy_HSV_Eu = cal_acc(vector_HSV, test_HSV, y_test_label, "E")
confusion_matrix_HSV_Pe, accuracy_HSV_Pe = cal_acc(vector_HSV, test_HSV, y_test_label, "P")

print "----------------------------- Result for Euclidean distance -----------------------------"
print confusion_matrix_HSV_Eu
print "Accuracy is " + str(accuracy_HSV_Eu) + "%"

print "----------------------------- Result for Pearson correlation coefficient -----------------------------"
print confusion_matrix_HSV_Pe
print "Accuracy is " + str(accuracy_HSV_Pe) + "%"

----------------------------- Result for Euclidean distance -----------------------------
[[183  22  47   5   3   4]
 [ 41 182  72  21   7  22]
 [ 43  19 161  10   4  13]
 [ 68  44  82 122  17  24]
 [ 32  33  89  38  53  16]
 [ 84  50 121  23  12  33]]
Accuracy is 40.78%
----------------------------- Result for Pearson correlation coefficient -----------------------------
[[176  28  42   3   5  10]
 [ 28 212  58   9  14  24]
 [ 33  37 139   6   8  27]
 [ 47  64  63 131  23  29]
 [ 26  55  53  23  68  36]
 [ 56  76 101  24  20  46]]
Accuracy is 42.89%


End